<a href="https://colab.research.google.com/github/Kesava-G/Kesava-G/blob/main/RSI_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoo_fin
!pip install pandas
!pip install ta
!pip install numpy

In [ ]:
import yahoo_fin.stock_info as yf
import pandas as pd
from ta import add_all_ta_features
from ta.momentum import RSIIndicator
from datetime import datetime
import numpy as np

In [ ]:
data = yf.get_data("aapl")

In [ ]:
#Get all technical indicators
rsi_14 = RSIIndicator(close=data.close, window=14)
data['RSI_14_days'] = rsi_14.rsi()

In [ ]:
#Buy when it is below 30 & Sell when its above 70
data = data.iloc[13:][:]

In [ ]:
#Function to indicate close price to buy and sell
def buy_sell(dataframe):
  Buy= []
  Sell = []
  flag = -1
  for i in range(0, len(dataframe)):
    if dataframe['RSI_14_days'][i]<30:
      Sell.append(np.nan)
      if flag != 1:
        Buy.append(dataframe['close'][i])
        flag = 1
      else:
        Buy.append(np.nan)
    elif dataframe['RSI_14_days'][i]>70:
      Buy.append(np.nan)
      if flag != 0:
        Sell.append(dataframe['close'][i])
        flag = 0
      else:
        Sell.append(np.nan)
    else:
      Buy.append(np.nan)
      Sell.append(np.nan)
  return (Buy, Sell)

In [ ]:
#buy and sell signals
b = buy_sell(data)
data['Buy'] = b[0]
data["Sell"] = b[1]
BuyFilter = list(filter(lambda x: not pd.isna(x), data['Buy']))
SellFilter = list(filter(lambda x: not pd.isna(x), data['Sell']))
if len(BuyFilter) > len(SellFilter):
  del BuyFilter[-1]
if len(SellFilter) > len(BuyFilter):
  del SellFilter[-1]
#create profit data frame
profitfortrade = {'Trade':[], "Profit":[]}
for trade in range(len(SellFilter)): 
  profitfortrade['Trade'].append(trade+1)
  profitfortrade['Profit'].append(SellFilter[trade]-BuyFilter[trade])
profitfortrade = pd.DataFrame(data=profitfortrade)
profitfortrade = profitfortrade.style.hide_index()
profitfortrade

Trade,Profit
1,0.043527
2,-0.013950
3,-0.005581
4,-0.084822
5,-0.021206
6,0.089286
7,0.008929
8,0.010045
9,0.055804
10,0.020089
